In [1]:
%pylab inline
from functools import partial

import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

Populating the interactive namespace from numpy and matplotlib


# Part 1

## 1.1 Linear Regression

###   1.

In [ ]:
# Independent variable
x = [1, 2, 3, 4]

# Number of samples
m = len(x)

# Number of features
n = 1

# Build feature matrix
X = np.array(x)
X.shape = (m, n)

# Observations
y_train = np.array([3, 2, 0, 5])
y_train.shape = (1, m)

In [ ]:
title("Raw Data")
scatter(X, y_train)
show()

In [ ]:
def polynomial_feature_map(X, k=2):
    """
    Number of features, n = 1.
    """
    m, n = X.shape
    
    phi = np.zeros(shape=(m, k))
    
    for i, x in enumerate(X):
        mapped_feature = np.zeros(shape=(k))
        for j in range(k):
            mapped_feature[j] = x**j
        
        phi[i] = mapped_feature
    
    return np.matrix(phi)

In [ ]:
def linear_regression(Phi, y_train):
    """
    Apply normal equations to find weights
    """

    a = np.matrix(np.matmul(Phi.T, Phi))
    b = np.matrix(np.matmul((Phi.T), y_train.T))

    w = np.matmul(a.I, b)
    
    return w

In [ ]:
def polynomial(x, k, *w):
    """
    Apply polynomial weights derived from normal equations.
    """
    result = np.zeros_like(x)
    for i, sample in enumerate(x):
        res = 0
        for j in range(k):
            res += w[j]*sample**j
        result[i] = res

    return result

In [ ]:
# Plot raw data points
scatter(X, y_train)

# Test data
x_test = np.arange(0, 4.1, 0.1)

# Plot each model
for i in range(1, 5):
    Phi = polynomial_feature_map(X, k=i)
    w = linear_regression(Phi, y_train)
    plot(x_test, polynomial(x_test, len(w), *w), label=i)
    
title('Linear Regression With Different Feature Maps')
legend(title='$k$')

In [ ]:
# Weights for k=1 to k=3

for k in range(1, 4):
    Phi = polynomial_feature_map(X, k)
    w = linear_regression(Phi, y_train)
    print(f"k={k}")
    print("**********")
    print(w)

In [28]:
def sse(y, y_hat):        
    return sum((y-y_hat)**2)

def mse(y, y_hat):
    return sse(y, y_hat)/len(y_hat)

In [ ]:
error_vec = []

for i in range(1, 5):
    Phi = polynomial_feature_map(X, k=i)
    w = linear_regression(Phi, y_train)
    error_vec.append(mse(y_train, polynomial(x, len(w), *w)))

title("MSE vs k")
plot(range(1, 5), error_vec, '-o')
xlabel('k')
ylabel('Error')
show()

###   2.

In [ ]:
def g_sigma(x, sigma):
    epsilon = np.random.normal(scale=sigma, size=x.size)
    return np.sin(2*np.pi*x)**2 + epsilon

def exact(x):
    return np.sin(2*np.pi*x)**2

In [ ]:
x_sample_30 = np.random.random(size=30)
x = np.arange(0, 1, 0.01)
g_sigma_noise = partial(g_sigma, sigma=0.07)
y_sample_30 = g_sigma_noise(x_sample_30)

In [ ]:
title("Raw Data points and Exact Solution")
scatter(x_sample_30, y_sample_30)
plot(x, exact(x))
show()

In [ ]:
m = len(x_sample_30)
n = 1

X_train = np.array(x_sample_30)
X_train.shape = (m, n)
y_train = y_sample_30
y_train.shape = (1, m)

In [ ]:
k_vector = [2, 5, 10, 14, 18]

for k in k_vector:
    figure()
    title(f'k={k}')
    Phi = polynomial_feature_map(X_train, k)
    w = linear_regression(Phi, y_train)
    scatter(x_sample_30, polynomial(x_sample_30, len(w), *w), label=k)
    scatter(x_sample_30, y_sample_30)
ylim((-1.5, 1.5))
show()

In [ ]:
m = len(x_sample_30)
n = 1

X_train = np.array(x_sample_30)
X_train.shape = (m, n)
y_train = y_sample_30
y_train.shape = (1, m)

training_error_vec = []
k_vector = [2, 5, 10, 14, 18]
for k in k_vector:
    Phi = polynomial_feature_map(X_train, k)
    w = linear_regression(Phi, y_train)
    error = mse(y_sample_30, polynomial(x_sample_30, len(w), *w))
    training_error_vec.append(np.log(error))

title("Training Error Vs $k$")
plot(k_vector, training_error_vec, '-o')
xlabel('k')
ylabel('ln(Error)')
show()

The increasing error with K is due to the ill-conditioned matrix for large K values

In [ ]:
x_sample_1000 = np.random.random(size=1000)
g_partial = partial(g_sigma, sigma=0.07)
y_sample_1000 = g_partial(x_sample_1000)

scatter(x_sample_1000, y_sample_1000)

x = np.arange(0, 1, 0.01)
y = np.sin(2*np.pi*x)**2

plot(x, y, color='orange')
title("Sample of 1000 points")
show()

In [ ]:
m = len(x_sample_1000)
n = 1

X_test = np.array(x_sample_1000)
X_test.shape = (m, n)
y_test = y_sample_1000
y_test.shape = (1, m)

In [ ]:
test_error_vec = []
k_vector = arange(2,18,1)

for k in k_vector:
    Phi = polynomial_feature_map(X_train, k)
    w = linear_regression(Phi, y_train)
    error = mse(y_test, polynomial(x_sample_1000, len(w), *w))
    test_error_vec.append(np.log(error))

In [ ]:
title("Test Error Vs $k$")
plot(k_vector, test_error_vec, '-o')
ylabel("ln(error)")
xlabel("$k$")
show()

In [ ]:
k_vector = [2, 5, 10, 14, 18]

for k in k_vector:
    figure()
    title(f'k={k}')
    Phi = polynomial_feature_map(X_train, k)
    w = linear_regression(Phi, y_train)

    scatter(x_sample_1000, polynomial(x_sample_1000, len(w), *w), label=k)
    scatter(x_sample_1000, y_sample_1000)

    ylim((-1.5, 1.5))
show()

In [ ]:
def error_experiment(feature_map, basis, x_sample, y_sample, k_vector):
    m = len(x_sample)
    n = 1

    X = np.array(x_sample)
    X.shape = (m, n)
    y = y_sample
    y.shape = (1, m)

    error_vec = []
    for k in k_vector:
        Phi = feature_map(X, k)
        w = linear_regression(Phi, y)
        error = mse(y_sample, basis(x_sample, len(w), *w))
        error_vec.append(error)
    
    return error_vec

def runner(feature_map, basis, x_sample, k_vector, nruns=5):
    """
    Function runner
    """
    
    results = []
    for i in range(nruns):
        y_sample = g_sigma_noise(x_sample)
        results.append(error_experiment(feature_map, basis, x_sample, y_sample, k_vector))
    return np.array(results)


In [ ]:
train_results = runner(polynomial_feature_map, polynomial, x_sample_30, [2, 5, 10, 14, 18])

In [ ]:
title("Train Error (100 Runs)")
plot(np.log(train_results.mean(axis=0)), '-o')
xlabel('k')
ylabel('Mean of ln(error)')

In [ ]:
test_results = runner(polynomial_feature_map, polynomial, x_sample_1000, arange(2, 18, 1))

In [ ]:
title("Test Error (100 Runs)")
plot(np.log(test_results.mean(axis=0)), '-o')
xlabel('k')
ylabel('Mean of ln(error)')

### 3.

In [ ]:
def sinusoidal_feature_map(X, k=2):
    """
    Number of features, n = 1.
    """
    m, n = X.shape
    
    phi = np.zeros(shape=(m, k))
    
    for i, x in enumerate(X):
        mapped_feature = np.zeros(shape=(k))
        for j in range(k):
            mapped_feature[j] = np.sin((j+1)*np.pi*x)
        
        phi[i] = mapped_feature
    
    return np.matrix(phi)

In [ ]:
def sinusoidal(x, k, *w):
    """
    Apply sinusoidal weights derived from normal equations.
    """
    result = np.zeros_like(x)
    for i, sample in enumerate(x):
        res = 0
        for j in range(k):
            res += w[j]*np.sin((j+1)*np.pi*sample)
        result[i] = res

    return result

In [ ]:
k_vector = arange(1, 18, 1)

for k in k_vector:
    figure()
    title(f'k={k}')
    Phi = sinusoidal_feature_map(X_train, k)
    w = linear_regression(Phi, y_train)
    scatter(x_sample_30, sinusoidal(x_sample_30, len(w), *w), label=k)
    scatter(x_sample_30, y_sample_30)
ylim((-1.5, 1.5))
show()

In [ ]:
train_results = runner(sinusoidal_feature_map, sinusoidal, x_sample_30, [2, 5, 10, 14, 18])

In [ ]:
title("Train Error (100 Runs)")
plot(np.log(train_results.mean(axis=0)), '-o')
xlabel('k')
ylabel('Mean of ln(error)')

In [ ]:
test_results = runner(sinusoidal_feature_map, sinusoidal, x_sample_1000, arange(2, 18, 1))

In [ ]:
title("Test Error (100 Runs)")
plot(np.log(test_results.mean(axis=0)), '-o')
xlabel('k')
ylabel('Mean of ln(error)')

 ## 1.2 Boston housing and kernels

#### Data Preparation

In [ ]:
# Load data
boston_dataset = sio.loadmat('boston.mat')
X, y = boston_dataset['boston'][:,:-1], boston_dataset['boston'][:, -1: ]

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

m, n = X_train.shape

### 4.a Fitting with a constant function

Intuitively just finding the mean of the dataset.

In [ ]:
training_ones = np.ones_like(y_train)
test_ones = np.ones_like(y_test)

In [ ]:
training_ones.shape

In [ ]:
w = linear_regression(training_ones, y_train.T)
plot(y_train, 'o', label='House Prices')
plot(polynomial(X_train, len(w), *w), 'o')
legend()
ylabel('Price in $1000s')
show()

In [ ]:
mse_train = mse(y_train, polynomial(X_train, len(w), *w))
mse_test = mse(y_test, polynomial(X_test, len(w), *w))

In [ ]:
print(f"MSE on the training data is {mse_train}")
print(f"MSE on the test data is {mse_test}")

In [ ]:
np.matmul(X.T,X).shape

#### b.

This is like finding the mean of the dependent variable data, or just the bias term.

#### c.

In [ ]:
def identity_feature_map(X, k=None):
    """
    Identity feature map, add a bias term.
    """
   
    m, n = X.shape
    k = n + 1

    phi = np.zeros(shape=(m, k))
    
    for i, x in enumerate(X):
        mapped_feature = np.zeros(shape=(k))
        for j in range(k-1):
            mapped_feature[j] = x[j]
        
        # Add extra bias feature
        mapped_feature[-1] = 1
        
        phi[i] = mapped_feature
    
    return np.matrix(phi)
    

In [ ]:
def identity_predictor(x, w):
    return np.dot(x, w)

In [ ]:
# pick out nth feature
for n in range(1,13):

    # Reshape, and use identity feature map to add bias term
    feature_n = X_train[:,n].reshape(len(X_train), 1)
    mapped_feature_n = identity_feature_map(feature_n)
    w = linear_regression(mapped_feature_n, y_train.T)
    mapped_feature_n_test = identity_feature_map(X_test[:,n].reshape(len(X_test), 1))
    predictions = [float(identity_predictor(feat, w)) for feat in mapped_feature_n_test]
    mean_square_error =  mse(y_train, predictions)
    print(f"MSE with feature {n} {mean_square_error}")

In [ ]:
w_all_attributes = linear_regression(identity_feature_map(X_train), y_train.T)

In [ ]:
predictions_all_attributes = [float(identity_predictor(feat, w_all_attributes)) for feat in identity_feature_map(X_test)]
plot(y_test, 'o', label='House Prices')
plot(predictions_all_attributes, 'o')


In [ ]:
mean_square_error =  mse(y_test, np.array(predictions_all_attributes))
print(f"MSE with feature all attributes {mean_square_error}")

## 1.3 Kernelised Ridge Regression

In [2]:
def gaussian_kernel(xi, xj, sigma=0.1):
    tmp = np.linalg.norm(x=(xi-xj),ord=2)
    return np.exp((-tmp**2)/(2*sigma**2))

In [14]:
def kernel_regression(X_train, y_train, m, sigma, gamma):
    K = np.zeros(shape=(m, m))

    for i in range(0, m):
        for j in range(0, m):
            K[i][j] = gaussian_kernel(X_train[i], X_train[j], sigma)

    K = np.matrix(K)
    alpha_opt = np.matmul((K+(gamma*m*np.identity(m))).I, y_train)

    return alpha_opt

In [15]:
def kernel_predictor(X_train, X_test, alpha_opt, sigma):
    
    m, n = X_train.shape
    y_test = []

    for i, x_test in enumerate(X_test):
        res = 0
        for j in range(m):
            res += float(alpha_opt[j]*gaussian_kernel(x_test, X_train[j], sigma))
        y_test.append(res)

    return np.array(y_test)

#### 5.a

In [132]:
# Load data
boston_dataset = sio.loadmat('boston.mat')
X, y = boston_dataset['boston'][:,:-1], boston_dataset['boston'][:, -1: ]

# Train/Test Split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.33, random_state=42)

m, n = X_train.shape

In [133]:
gamma_vector = np.array(
    [2**(i) for i in range(-40, -25, 1)]
)

sigma_vector = np.array(
    [2**(i) for i in linspace(7,13,13)]
)

In [104]:
# Learn parameters
sigma = sigma_vector[0]
gamma = gamma_vector[2]

alpha_opt = kernel_regression(X_train, y_train, m, sigma, gamma)

In [105]:
results = kernel_predictor(X_train, X_test, alpha_opt, sigma)

#### Cross validation

In [141]:
parameters = [
    (sigma, gamma) 
    for sigma in sigma_vector
    for gamma in gamma_vector
]
shuffle(parameters)

In [143]:
print(f"There are {len(parameters)} parameter combinations")

There are 195 parameter combinations


In [144]:
def kfold_cross_validation(X, y, sigma, gamma, k=5):
    
    kf = KFold(n_splits=k)
    errors = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        m, n = X_train.shape

        alpha_opt = kernel_regression(
            X_train, y_train, m, sigma, gamma)
        

        y_hat = kernel_predictor(
            X_train, X_test, alpha_opt, sigma)

        error = sse(y_test, y_hat)
        errors.append(error)
    
    return np.array(errors).mean()

#### Run cross validation experiments over all parameter combinations

In [145]:
cross_validation_results = []

for (s, g) in parameters[:9]:
    cross_validation_results.append(
        (s, g, kfold_cross_validation(X, y, s, g))
    )

    
# Convert to array for ease of indexing
cross_validation_results  = np.array(cross_validation_results)

In [146]:
cross_validation_results

array([[7.24077344e+02, 7.45058060e-09, 2.01633870e+06],
       [5.79261875e+03, 1.86264515e-09, 1.17673954e+06],
       [7.24077344e+02, 3.72529030e-09, 2.28072059e+06],
       [4.09600000e+03, 9.31322575e-10, 1.24259167e+06],
       [1.28000000e+02, 2.91038305e-11, 8.29778411e+08],
       [8.19200000e+03, 3.72529030e-09, 1.10162399e+06],
       [4.09600000e+03, 2.32830644e-10, 1.24187972e+06],
       [1.81019336e+02, 7.27595761e-12, 9.25741773e+08],
       [1.28000000e+02, 3.63797881e-12, 3.45613041e+09]])

In [167]:
gammas, sigmas, cv_error = (
    cross_validation_results[:,0:1],
    cross_validation_results[:,1:2],
    cross_validation_results[:,2:]
)